In [ ]:
import requests

url = "https://api.steampowered.com/ISteamApps/GetAppList/v2/"
response = requests.get(url)
data = response.json()
apps = data["applist"]["apps"]
data

In [ ]:
appid = 150
details_url = f"https://store.steampowered.com/api/appdetails?appids={appid}"
details_resp = requests.get(details_url)
details_data = details_resp.json()

print(details_data)
if details_data[str(appid)]['success']:
    app_info = details_data[str(appid)]["data"]
    if app_info.get('type') == 'game':
        print('게임입니다')
    else : 
        print('제외대상입니다')

In [1]:
import pandas as pd
import re
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm

# 1. CSV 불러오기
df = pd.read_csv("스팀게임파일.csv")

# 2. 외국어 감지 함수 정의
def contains_cyrillic(text):
    if pd.isna(text):
        return False
    return bool(re.search('[\u0400-\u04FF]', str(text)))

def mostly_english(text):
    if pd.isna(text):
        return False
    text = str(text)
    korean_chars = len(re.findall('[가-힣]', text))
    english_chars = len(re.findall('[A-Za-z]', text))
    return english_chars > 20 and korean_chars < 5

# 3. 러시아어 또는 영어 비중 높은 항목 필터링
mask_russian = df[["genres", "release_date", "description_kr"]].applymap(contains_cyrillic).any(axis=1)
mask_english = df["description_kr"].apply(mostly_english)
mask_foreign = mask_russian | mask_english
foreign_indexes = df[mask_foreign].index

# 4. Steam 페이지에서 한국어 정보 크롤링 함수
def get_korean_info_from_steam(appid):
    url = f"https://store.steampowered.com/app/{appid}/?l=koreana"
    headers = {'User-Agent': 'Mozilla/5.0'}
    try:
        resp = requests.get(url, headers=headers, timeout=10)
        soup = BeautifulSoup(resp.text, "html.parser")

        name = soup.select_one(".apphub_AppName")
        description = soup.select_one("#game_area_description")
        date = soup.select_one(".release_date .date")
        genre_tags = soup.select('div.details_block a[href*="genre"]')

        return {
            "name_kr": name.get_text(strip=True) if name else None,
            "description_kr": description.get_text(strip=True) if description else None,
            "release_date": date.get_text(strip=True) if date else None,
            "genres": ", ".join([tag.text.strip() for tag in genre_tags]) if genre_tags else None
        }
    except Exception as e:
        print(f"[{appid}] 크롤링 실패: {e}")
        return {}

# 5. 필터된 게임에 대해 크롤링하여 덮어쓰기
for idx in tqdm(foreign_indexes, desc="Steam 한국어 페이지로 업데이트 중"):
    appid = df.loc[idx, "appid"]
    new_data = get_korean_info_from_steam(appid)
    for col in ["name_kr", "description_kr", "genres", "release_date"]:
        if new_data.get(col):  # 새 값이 있으면 덮어쓰기
            df.at[idx, col] = new_data[col]

# 6. 결과 저장
df.to_csv("스팀게임파일_한글정제완료.csv", index=False, encoding="utf-8-sig")
print("✅ 완료: 외국어 포함 항목을 한국어 페이지 기준으로 덮어씌웠습니다.")


C:\Users\main\AppData\Local\Temp\ipykernel_12836\428924196.py:25: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  mask_russian = df[["genres", "release_date", "description_kr"]].applymap(contains_cyrillic).any(axis=1)
Steam 한국어 페이지로 업데이트 중: 100%|██████████| 9732/9732 [1:58:05<00:00,  1.37it/s]  


✅ 완료: 외국어 포함 항목을 한국어 페이지 기준으로 덮어씌웠습니다.
